# Загрузка и обработка данных huggingface datasets
## Полезные ссылки:
 - классы и функции бибилиотеки datasets и https://huggingface.co/docs/datasets/v2.13.1/en/package_reference/main_classes
 - база датасетов https://huggingface.co/datasets
 - как устроены датасеты и что такое Memory-mapping https://huggingface.co/docs/datasets/about_arrow
 - разница iterable и map датасетов https://huggingface.co/docs/datasets/about_mapstyle_vs_iterable
 - быстрый гайд по обработке данных https://huggingface.co/docs/datasets/process
 - обработка батчами https://huggingface.co/docs/datasets/about_map_batch

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
import datasets
import transformers

# proxy
os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

/home/posokhov@ad.speechpro.com/anaconda3/envs/pl_template2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = datasets.load_dataset(
    "merionum/ru_paraphraser", # dataset name
    cache_dir="/mnt/cs/nlu/home/posokhov/" # path to savedir
    )
print(ds)
ds['train'].to_pandas()

Using custom data configuration merionum--ru_paraphraser-e39dafb2b050eb83
Found cached dataset json (/mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 2/2 [00:00<00:00, 129.71it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})


,id,id_1,id_2,text_1,text_2,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0
...,...,...,...,...,...,...
7222,25514,34622,34633,Путин освободил от должности ряд генералов,Путин снял с должностей более 20 руководителей...,0
7223,25524,34566,34654,Облака над Москвой в День Победы разгонят девя...,Путеводитель по Дню Победы: как провести 9 мая...,-1
7224,25548,34519,34681,Любляна отпразднует День Победы вместе с Москвой,В Москве ограничат движение в связи с Днем Победы,-1
7225,25549,34565,34681,Девять самолетов ВВС разгонят облака над Москв...,В Москве ограничат движение в связи с Днем Победы,-1


In [3]:
ds = ds.remove_columns(['id', 'id_1', 'id_2'])
ds = ds.rename_columns({"class":"label"})
ds

DatasetDict({
    train: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 1924
    })
})

In [4]:
def re_labeler(label, label_dict):
    new_label = label_dict[label]
    return {"label": new_label}


label_dict = {"-1": 0, "0": 1, "1": 1}

ds = ds.map(
    re_labeler,
    input_columns=["label"],
    fn_kwargs={"label_dict": label_dict},
    num_proc=4,
)
ds

Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-6e3ee3f0501577fa.arrow
Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-70fd87221a93b84e.arrow
Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-87e228d025f952e8.arrow
Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-7a517edbfbd43a8d.arrow
Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/

DatasetDict({
    train: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 1924
    })
})

In [5]:
ds_pos_pairs = ds["train"].filter(lambda label: label==1, input_columns=["label"])
ds_pos_pairs

Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-2dbd75ad37431229.arrow


Dataset({
    features: ['text_1', 'text_2', 'label'],
    num_rows: 4645
})

In [6]:
def map_neg_sampler(batch):
    text_1 = batch["text_1"][0]
    text_2 = batch["text_2"][1]
    return {
        "text_1":[text_1],
        "text_2":[text_2],
        "label": [0]
    }
ds_neg_pairs = ds_pos_pairs.map(map_neg_sampler, batched=True, batch_size=5)
ds_neg_pairs

Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-164637f1ceebb0fd.arrow


Dataset({
    features: ['text_1', 'text_2', 'label'],
    num_rows: 929
})

In [7]:
# merge train split
new_train = datasets.concatenate_datasets([ds['train'], ds_neg_pairs])
ds['train'] = new_train
ds = ds.shuffle(seed=42)
ds

Loading cached shuffled indices for dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-3c6e3ee315f7689a.arrow
Loading cached shuffled indices for dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-24c5bbd65b86a7db.arrow


DatasetDict({
    train: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 8156
    })
    test: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 1924
    })
})

In [8]:
ds['val'] = ds['train'].train_test_split(1000, seed=42)['test']

ds_hand_test = {"text_1":["сегодня ожидается ясная, теплая погода"], "text_2":["сегодня будет ясно и тепло"]}
ds_hand_test = datasets.Dataset.from_dict(ds_hand_test)
ds['ds_hand_test'] =ds_hand_test

ds

DatasetDict({
    train: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 8156
    })
    test: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 1924
    })
    val: Dataset({
        features: ['text_1', 'text_2', 'label'],
        num_rows: 1000
    })
    ds_hand_test: Dataset({
        features: ['text_1', 'text_2'],
        num_rows: 1
    })
})

In [9]:
# concat text_1 and text_2
def map_concat(text_1, text_2, sep_token="[SEP_TOKEN]"):
    return {"pair": sep_token.join([text_1, text_2])}
ds = ds.map(map_concat, input_columns=["text_1", "text_2"], remove_columns=["text_1", "text_2"])
ds

Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-0e0744a56853d46f.arrow
Loading cached processed dataset at /mnt/cs/nlu/home/posokhov/merionum___json/merionum--ru_paraphraser-e39dafb2b050eb83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-bd530113c166771c.arrow
100%|██████████| 1/1 [00:00<00:00, 2131.25ex/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'pair'],
        num_rows: 8156
    })
    test: Dataset({
        features: ['label', 'pair'],
        num_rows: 1924
    })
    val: Dataset({
        features: ['label', 'pair'],
        num_rows: 1000
    })
    ds_hand_test: Dataset({
        features: ['pair'],
        num_rows: 1
    })
})

In [10]:
ds.save_to_disk("/mnt/cs/nlu/home/posokhov/new_merionum")